In [1]:
import findspark
findspark.init('C:/spark/spark-2.4.5-bin-hadoop2.7')

from pyspark.sql import SparkSession
import pyspark

In [2]:
spark = SparkSession.builder.appName('churn_logreg').getOrCreate()

In [3]:
from pyspark.ml.classification import LogisticRegression

In [4]:
df = spark.read.csv('customer_churn.csv', inferSchema=True, header=True)

In [5]:
df.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [7]:
df.describe().show()

+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|summary|        Names|              Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|            Location|             Company|              Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|  count|          900|              900|              900|               900|              900|               900|                 900|                 900|                900|
|   mean|         null|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777|                null|                null|0.16666666666666666|
| stddev|         null|6.127560416916251|2408.644531858096|0.4999208935073339|1.274449013194616|1.764835592035

In [8]:
df.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [9]:
from pyspark.ml.feature import VectorAssembler

In [10]:
assembler = VectorAssembler(inputCols=['Age', 'Total_Purchase', 
                                       'Account_Manager', 'Years',
                                       'Num_Sites'], 
                            outputCol='features')

In [11]:
output = assembler.transform(df)

In [13]:
output.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)
 |-- features: vector (nullable = true)



In [14]:
final_data = output.select('features', 'Churn')

In [15]:
final_data.show()

+--------------------+-----+
|            features|Churn|
+--------------------+-----+
|[42.0,11066.8,0.0...|    1|
|[41.0,11916.22,0....|    1|
|[38.0,12884.75,0....|    1|
|[42.0,8010.76,0.0...|    1|
|[37.0,9191.58,0.0...|    1|
|[48.0,10356.02,0....|    1|
|[44.0,11331.58,1....|    1|
|[32.0,9885.12,1.0...|    1|
|[43.0,14062.6,1.0...|    1|
|[40.0,8066.94,1.0...|    1|
|[30.0,11575.37,1....|    1|
|[45.0,8771.02,1.0...|    1|
|[45.0,8988.67,1.0...|    1|
|[40.0,8283.32,1.0...|    1|
|[41.0,6569.87,1.0...|    1|
|[38.0,10494.82,1....|    1|
|[45.0,8213.41,1.0...|    1|
|[43.0,11226.88,0....|    1|
|[53.0,5515.09,0.0...|    1|
|[46.0,8046.4,1.0,...|    1|
+--------------------+-----+
only showing top 20 rows



In [16]:
train_churn, test_churn = final_data.randomSplit([0.7, 0.3])

In [17]:
from pyspark.ml.classification import LogisticRegression

In [19]:
lr_churn = LogisticRegression(labelCol='Churn')

In [20]:
fitted_churn_model = lr_churn.fit(train_churn)

In [21]:
training_sum = fitted_churn_model.summary

In [27]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [28]:
pred_and_labels = fitted_churn_model.evaluate(test_churn)

In [33]:
pred_and_labels.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|Churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[22.0,11254.38,1....|    0|[5.40717132354713...|[0.99553571229502...|       0.0|
|[28.0,8670.98,0.0...|    0|[9.11702108707470...|[0.99989023083117...|       0.0|
|[28.0,9090.43,1.0...|    0|[1.83397098882586...|[0.86223410536625...|       0.0|
|[28.0,11204.23,0....|    0|[1.91110878687204...|[0.87114366304823...|       0.0|
|[29.0,8688.17,1.0...|    1|[3.23691055654612...|[0.96219990288547...|       0.0|
|[29.0,9617.59,0.0...|    0|[5.28405998548266...|[0.99495379581446...|       0.0|
|[29.0,12711.15,0....|    0|[6.28482938776324...|[0.99813909439809...|       0.0|
|[29.0,13240.01,1....|    0|[7.68924650558373...|[0.99954248648092...|       0.0|
|[30.0,7960.64,1.0...|    1|[3.39169941037112...|[0.96744411186592...|       0.0|
|[30.0,8403.78,1

In [34]:
churn_eval = BinaryClassificationEvaluator(labelCol='Churn')

In [35]:
auc = churn_eval.evaluate(pred_and_labels.predictions)

In [36]:
auc

0.8542647335718312

In [37]:
final_lr_churn_model = lr_churn.fit(final_data)

In [38]:
new_customers = spark.read.csv('new_customers.csv', inferSchema=True, header=True)

In [39]:
new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)



In [40]:
test_new_customers = assembler.transform(new_customers)

In [41]:
test_new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- features: vector (nullable = true)



In [42]:
final_results = final_lr_churn_model.transform(test_new_customers)

In [44]:
final_results.select('Company', 'prediction').show()

+----------------+----------+
|         Company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+

